In [1]:
import pandas as pd 
import numpy as np
import yfinance as yf
from pandas_datareader import data

In [21]:
df = yf.download('PTBA.jk',
                 start='2014-12-31', 
                 end='2019-12-31',
                 actions='inline',
                 progress=False)

df = df.loc[:, ['Adj Close']]
df.rename(columns={'Adj Close':'adj_close'}, inplace=True)

In [22]:
df['simple_rtn'] = df.adj_close.pct_change()
df['log_rtn'] = np.log(df.adj_close/df.adj_close.shift(1))

In [23]:
df.head()

,adj_close,simple_rtn,log_rtn
Date,,,
2015-01-02,1090.897461,NaN,NaN
2015-01-05,1088.693604,-0.002020,-0.002022
2015-01-06,1064.451538,-0.022267,-0.022519
2015-01-07,1066.655396,0.002070,0.002068
2015-01-08,1040.209351,-0.024793,-0.025106


In [24]:
import pandas as pd
import quandl
import seaborn as sns


QUANDL_KEY = '{key}' # replace {key} with your own API key  
quandl.ApiConfig.api_key = QUANDL_KEY

In [25]:
df_all_dates = pd.DataFrame(index=pd.date_range(start='2014-12-31', 
                                                end='2019-12-31'))

df = df_all_dates.join(df[['adj_close']], how='left') \
                 .fillna(method='ffill') \
                 .asfreq('A')

In [26]:
df_all_dates.head()

""
2014-12-31
2015-01-01
2015-01-02
2015-01-03
2015-01-04


In [27]:
df.head()

,adj_close
2014-12-31,NaN
2015-12-31,467.380341
2016-12-31,1590.298950
2017-12-31,1763.460815
2018-12-31,3393.033447


In [28]:
df_cpi = quandl.get("ODA/IDN_PCPI", 
                    authtoken="et7jqG6esE8zbud7VEaM", 
                    start_date='2014-12-31', 
                    end_date='2019-12-31')
df_cpi.rename(columns={'Value':'cpi'}, inplace=True)

In [29]:
df_cpi.head()

,cpi
Date,
2014-12-31,113.218
2015-12-31,120.422
2016-12-31,124.668
2017-12-31,129.416
2018-12-31,133.555


In [30]:
df_merged = df.join(df_cpi, how='right')

In [31]:
df_merged.head()

,adj_close,cpi
Date,,
2014-12-31,NaN,113.218
2015-12-31,467.380341,120.422
2016-12-31,1590.298950,124.668
2017-12-31,1763.460815,129.416
2018-12-31,3393.033447,133.555


In [32]:
df_merged['simple_rtn'] = df_merged.adj_close.pct_change()
df_merged['inflation_rate'] = df_merged.cpi.pct_change()

In [33]:
df_merged.head(10)

,adj_close,cpi,simple_rtn,inflation_rate
Date,,,,
2014-12-31,NaN,113.218,NaN,NaN
2015-12-31,467.380341,120.422,NaN,0.063629
2016-12-31,1590.298950,124.668,2.402580,0.035259
2017-12-31,1763.460815,129.416,0.108886,0.038085
2018-12-31,3393.033447,133.555,0.924076,0.031982
2019-12-31,2305.552246,138.022,-0.320504,0.033447


In [34]:
df_merged['real_rtn'] = (df_merged.simple_rtn + 1 ) / (df_merged.inflation_rate + 1) - 1
df_merged.head(10)

,adj_close,cpi,simple_rtn,inflation_rate,real_rtn
Date,,,,,
2014-12-31,NaN,113.218,NaN,NaN,NaN
2015-12-31,467.380341,120.422,NaN,0.063629,NaN
2016-12-31,1590.298950,124.668,2.402580,0.035259,2.286693
2017-12-31,1763.460815,129.416,0.108886,0.038085,0.068204
2018-12-31,3393.033447,133.555,0.924076,0.031982,0.864447
2019-12-31,2305.552246,138.022,-0.320504,0.033447,-0.342496


In [35]:
df = df_merged['adj_close'].dropna()
df.head(10)

Date
2015-12-31     467.380341
2016-12-31    1590.298950
2017-12-31    1763.460815
2018-12-31    3393.033447
2019-12-31    2305.552246
Name: adj_close, dtype: float64

In [36]:
x = df[0]
print(x)
y = df[-1]
print(y)

year = len(df)
print(year)

inflation_rate = df_merged['inflation_rate'].sum()
print(inflation_rate)    

avg_inflation_rate = inflation_rate/year 
print(avg_inflation_rate)

CAGR =(y/x)**(1/year)- 1 - avg_inflation_rate

print(CAGR)

467.3803405761719
2305.55224609375
5
0.20240298004550472
0.04048059600910094
0.3355272349274284
